# Spracovanie dát

Tento notebook slúži na kontrolu kvality raw dát a ich predspracovanie.

In [220]:
import pandas as pd
import numpy as np

In [221]:
cycle_data_raw = pd.read_csv('../Data/cyklotrasy_data_bratislava_raw.csv')
weather_data_raw = pd.read_csv('../Data/pocasie_data_raw.csv')

## Kvalita

Ako prvé skrátime dáta o cyklotrasách na časový rozsah dát o počasí.


In [222]:
cycle_data_raw["datetime"] = (
    pd.to_datetime(cycle_data_raw["attributes.DATUM_A_CAS"], utc=True)
      .dt.tz_convert("Europe/Bratislava")
      .dt.tz_localize(None)
)

weather_data_raw["date"] = pd.to_datetime(weather_data_raw["date"])

cycle_data_raw = cycle_data_raw[
    cycle_data_raw["datetime"].between(
        weather_data_raw["date"].min(),
        weather_data_raw["date"].max() + pd.Timedelta(days=0.9999)
    )
].copy()

Pozrieme sa na unikátne kombinácie cyklotrias. Vidíme, že je v niektorých názvoch a smeroch chyba - '\n' navyše. Na základe toho údaje upravíme.

In [223]:
distinct_combinations = (
    cycle_data_raw[["attributes.NAZOV", "attributes.SMER_DO", "attributes.SMER_Z"]]
    .drop_duplicates()
    .sort_values(["attributes.NAZOV", "attributes.SMER_DO", "attributes.SMER_Z"])
)

display(distinct_combinations.sort_values(by=["attributes.NAZOV", "attributes.SMER_DO", "attributes.SMER_Z"]))

,attributes.NAZOV,attributes.SMER_DO,attributes.SMER_Z
190401,#1 - Starý Most,Centrum,Petržalka
55600,#10 - Dunajská,Centrum (Kamenné námestie),Mlynské nivy
0,#11 - Most Apollo,Centrum,Petržalka
235203,#12 - Železná studnička,Ku studničke,Od studničky
224403,#13 - Vajanského 1,Most SNP,NIE JE - v mieste je jednosmerná premávka
572861,#13 - Vajanského 1,Most SNP,NIE JE - v mieste je jednosmerná premávka\n
213600,#14 - Vajanského 2,Šafárikovo námestie,NIE JE - v mieste je jednosmerná premávka
573018,#14 - Vajanského 2,Šafárikovo námestie\n,NIE JE - v mieste je jednosmerná premávka\n
257600,#15 - Incheba Einsteinova,Einsteinova,Centrum
254006,#16 - Trenčianska,Miletičova,Dulovo Námestie


In [224]:
str_cols = cycle_data_raw.select_dtypes(include="object").columns

cycle_data_raw[str_cols] = cycle_data_raw[str_cols].apply(lambda col: col.str.strip())

Pozrieme sa na základné údaje o dátach, ako sú počty údajov v stĺpci a prehľad hodnôt.

In [225]:
display(cycle_data_raw.count())
display(weather_data_raw.count())

attributes.NAZOV              450301
attributes.ZEMEPISNA_SIRKA    450301
attributes.ZEMEPISNA_DLZKA    450301
attributes.SMER_DO            450301
attributes.SMER_Z             450301
attributes.DATUM_A_CAS        450301
attributes.POCET_DO           450301
attributes.POCET_Z            450301
attributes.ObjectId           450301
datetime                      450301
dtype: int64

date    1085
tavg    1085
tmin    1085
tmax    1085
prcp    1084
snow       0
wdir       0
wspd    1085
wpgt    1085
pres    1085
tsun     748
dtype: int64

In [226]:
cycle_data_raw.describe()

,attributes.ZEMEPISNA_SIRKA,attributes.ZEMEPISNA_DLZKA,attributes.POCET_DO,attributes.POCET_Z,attributes.ObjectId,datetime
count,450301.000000,450301.000000,450301.000000,450301.00000,450301.000000,450301
mean,48.149043,17.092158,12.107348,7.35377,318598.803971,2024-08-17 03:39:18.950124544
min,48.110442,16.969000,0.000000,0.00000,1.000000,2023-01-01 00:00:00
25%,48.136692,17.073272,0.000000,0.00000,115496.000000,2023-12-08 04:00:00
50%,48.140819,17.116058,2.000000,1.00000,233859.000000,2024-09-01 06:00:00
75%,48.162144,17.127928,14.000000,6.00000,580864.000000,2025-05-10 01:00:00
max,48.210650,17.138556,583.000000,645.00000,693456.000000,2025-12-20 23:00:00
std,0.025676,0.052929,23.829372,19.27489,232033.702638,NaN


In [227]:
weather_data_raw.describe()

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,1085,1085.000000,1085.000000,1085.000000,1084.000000,0.0,0.0,1085.000000,1085.000000,1085.000000,748.000000
mean,2024-06-26 00:00:00,12.521935,8.181659,16.956866,1.666697,NaN,NaN,12.416313,30.272258,1016.847926,267.161765
min,2023-01-01 00:00:00,-5.700000,-9.100000,-3.400000,0.000000,NaN,NaN,1.900000,11.100000,990.100000,0.000000
25%,2023-09-29 00:00:00,5.700000,2.000000,9.200000,0.000000,NaN,NaN,8.300000,24.100000,1011.700000,80.000000
50%,2024-06-26 00:00:00,12.300000,8.200000,16.600000,0.000000,NaN,NaN,11.400000,29.600000,1016.400000,237.000000
75%,2025-03-24 00:00:00,19.200000,14.100000,24.700000,1.025000,NaN,NaN,15.500000,35.200000,1021.800000,419.250000
max,2025-12-20 00:00:00,28.900000,23.900000,36.000000,37.300000,NaN,NaN,34.400000,72.200000,1043.300000,792.000000
std,NaN,8.104544,7.340852,9.297833,4.167819,NaN,NaN,5.387349,8.845390,8.208956,211.413720


Skontrolujeme počet 'null' hodnôt a duplikátov.

In [228]:
print('Null values in cycle path data:', cycle_data_raw.isnull().sum(), '\n')
print('Null values in weather data:', weather_data_raw.isnull().sum())

Null values in cycle path data: attributes.NAZOV              0
attributes.ZEMEPISNA_SIRKA    0
attributes.ZEMEPISNA_DLZKA    0
attributes.SMER_DO            0
attributes.SMER_Z             0
attributes.DATUM_A_CAS        0
attributes.POCET_DO           0
attributes.POCET_Z            0
attributes.ObjectId           0
datetime                      0
dtype: int64 

Null values in weather data: date       0
tavg       0
tmin       0
tmax       0
prcp       1
snow    1085
wdir    1085
wspd       0
wpgt       0
pres       0
tsun     337
dtype: int64


In [229]:
dupes_cycle = cycle_data_raw.duplicated(subset=["attributes.ObjectId"]).sum()
print("Duplicated cycle path rows:", dupes_cycle)

dupes_weather = weather_data_raw.duplicated(subset=["date"]).sum()
print("Duplicated weather rows:", dupes_weather)

Duplicated cycle path rows: 0
Duplicated weather rows: 0


Skontrolujeme, či existujú výpadky v sčítačoch.

In [230]:
df = cycle_data_raw.copy().sort_values(["attributes.NAZOV", "datetime"])

rows = []

for nazov, path_df in df.groupby("attributes.NAZOV", sort=False):

    path_df = path_df.drop_duplicates(subset="datetime").sort_values("datetime")
    if len(path_df) < 2:
        continue

    # Plný rozsah hodín pre danú trasu
    full_hours = pd.date_range(
        path_df["datetime"].min(),
        path_df["datetime"].max(),
        freq="h"
    )

    # Chýbajúce hodiny
    missing_hours = full_hours.difference(path_df["datetime"])
    if missing_hours.empty:
        continue

    # Pre každú chýbajúcu hodinu nájdeme predchádzajúci a nasledujúci záznam
    for missing in missing_hours:
        idx = path_df["datetime"].searchsorted(missing)

        before = path_df.iloc[idx - 1] if idx > 0 else None
        after  = path_df.iloc[idx] if idx < len(path_df) else None

        rows.append({
            "attributes.NAZOV": nazov,
            "missing_hour": missing,
            "before_datetime": None if before is None else before["datetime"],
            "after_datetime": None if after is None else after["datetime"],
            "before_pocet_do": None if before is None else before["attributes.POCET_DO"],
            "before_pocet_z":  None if before is None else before["attributes.POCET_Z"],
            "after_pocet_do":  None if after is None else after["attributes.POCET_DO"],
            "after_pocet_z":   None if after is None else after["attributes.POCET_Z"],
        })

missing_context = (
    pd.DataFrame(rows)
      .sort_values(["attributes.NAZOV", "missing_hour"])
      .reset_index(drop=True)
)

display(missing_context)

,attributes.NAZOV,missing_hour,before_datetime,after_datetime,before_pocet_do,before_pocet_z,after_pocet_do,after_pocet_z
0,#1 - Starý Most,2023-03-16 11:00:00,2023-03-16 10:00:00,2023-03-16 12:00:00,28,3,26,6
1,#1 - Starý Most,2023-03-26 02:00:00,2023-03-26 01:00:00,2023-03-26 03:00:00,4,1,2,0
2,#1 - Starý Most,2023-09-21 20:00:00,2023-09-21 19:00:00,2023-09-21 21:00:00,35,3,0,0
3,#1 - Starý Most,2024-03-31 02:00:00,2024-03-31 01:00:00,2024-03-31 03:00:00,0,0,0,1
4,#1 - Starý Most,2024-04-16 20:00:00,2024-04-16 19:00:00,2024-04-16 21:00:00,18,1,3,0
...,...,...,...,...,...,...,...,...
8564,Viedenska,2025-10-22 19:00:00,2025-10-22 04:00:00,2025-10-23 00:00:00,0,0,0,0
8565,Viedenska,2025-10-22 20:00:00,2025-10-22 04:00:00,2025-10-23 00:00:00,0,0,0,0
8566,Viedenska,2025-10-22 21:00:00,2025-10-22 04:00:00,2025-10-23 00:00:00,0,0,0,0
8567,Viedenska,2025-10-22 22:00:00,2025-10-22 04:00:00,2025-10-23 00:00:00,0,0,0,0


## Predspracovanie

### Cyklotrasy

Premenujeme stĺpce z 'attributes.*' na samotné názvy atribútov a vynecháme nepotrebné stĺpce -  ObjectId a datum_a_cas (už máme 'date' ktorý je v správnej časovej zóne voči počasiu).

In [231]:
cycle_data = cycle_data_raw.rename(columns={
    'attributes.NAZOV': 'nazov',
    'attributes.ZEMEPISNA_SIRKA': 'zemepisna_sirka',
    'attributes.ZEMEPISNA_DLZKA': 'zemepisna_dlzka',
    'attributes.SMER_DO': 'smer_do',
    'attributes.SMER_Z': 'smer_z',
    'attributes.POCET_DO': 'pocet_do',
    'attributes.POCET_Z': 'pocet_z'})

cycle_data = cycle_data.drop(columns=['attributes.ObjectId', 'attributes.DATUM_A_CAS'])

Pridáme chýbajúce úseky z výpadkov sčítačov. Pokiaľ je chýbajúci úsek dlhý 1-2 hodiny, doplníme údaje 'pocet_do' a 'pocet_z' na základe okolitých dát.

In [232]:
cycle_data = cycle_data.sort_values(["nazov", "datetime"]).copy()

static_columns = ["nazov", "zemepisna_sirka", "zemepisna_dlzka", "smer_do", "smer_z"]
count_columns = ["pocet_do", "pocet_z"]

def fill_missing_hours(path_df):
    path_df = path_df.drop_duplicates(subset=["datetime"]).sort_values("datetime")

    # Pridáme chýbajúce hodinové úseky
    hourly = (
        path_df.set_index("datetime")
               .resample("h")
               .asfreq()
    )

    # Vyplníme stĺpce so stálou hodnotou
    hourly[static_columns] = hourly[static_columns].ffill().bfill()

    # Zaznačíme, ktoré riadky chýbali PRED interpoláciou
    missing = hourly[count_columns].isna().any(axis=1)
    
    # Rozdelíme na úseky s chýbajúcimi dátami
    gap_id = missing.ne(missing.shift(fill_value=False)).cumsum()
    gap_size = missing.groupby(gap_id).transform("sum")

    # Doplňame len 1-2 hodinové medzery
    fill_mask = missing & gap_size.le(2)
    interpol = hourly[count_columns].interpolate(method="time")
    hourly.loc[fill_mask, count_columns] = interpol.loc[fill_mask, count_columns]

    # Vytvoríme flag pre riadky, ktoré boli doplnené a zaokrúhlime hodnoty
    hourly["was_imputed"] = fill_mask
    hourly.loc[fill_mask, count_columns] = hourly.loc[fill_mask, count_columns].round().astype("Int64")
    
    return hourly.reset_index()

cycle_data = (
    cycle_data.groupby("nazov", group_keys=False, sort=False)
              .apply(fill_missing_hours)
              .reset_index(drop=True)
)

C:\Users\krebe\AppData\Local\Temp\ipykernel_37900\1763389821.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fill_missing_hours)


In [233]:
def season_from_month(m):
    if m in [12, 1, 2]: return "winter"
    if m in [3, 4, 5]: return "spring"
    if m in [6, 7, 8]: return "summer"
    return "autumn"

Vytvoríme nové stĺpce, ktoré môžu byť užitočné na zodpovedanie výskumných otázok.

In [234]:
# Fix po imputácii
cycle_data["datetime"] = pd.to_datetime(cycle_data["datetime"])

cycle_data["pocet_total"] = cycle_data[["pocet_do", "pocet_z"]].sum(axis=1, min_count=2)
cycle_data["date"] = cycle_data["datetime"].dt.normalize()
cycle_data["weekday"] = cycle_data["datetime"].dt.weekday
cycle_data["is_weekend"] = cycle_data["weekday"].isin([5, 6]).astype(int)
cycle_data["month"] = cycle_data["datetime"].dt.month
cycle_data["year"] = cycle_data["datetime"].dt.year
cycle_data["season"] = cycle_data["month"].apply(season_from_month)

### Počasie

Vynecháme prázne dtĺpce a doplníme chýbajúce dáta.

In [235]:
weather_data = weather_data_raw.drop(columns=['snow', 'wdir'])

weather_data['prcp'] = weather_data['prcp'].fillna(0)

## Spojenie datasetov

Joinneme datasety na základe stĺpcov 'date'.

In [236]:
data = cycle_data.merge(weather_data, on='date', how='left')

print(data.shape)
print(data["tavg"].isna().sum())

(458870, 24)
0


In [237]:
data.describe()

,datetime,zemepisna_sirka,zemepisna_dlzka,pocet_do,pocet_z,pocet_total,date,weekday,is_weekend,month,year,tavg,tmin,tmax,prcp,wspd,wpgt,pres,tsun
count,458870,458870.000000,458870.000000,450425.000000,450425.000000,450425.000000,458870,458870.000000,458870.000000,458870.000000,458870.000000,458870.000000,458870.000000,458870.000000,458870.000000,458870.000000,458870.000000,458870.000000,337097.000000
mean,2024-08-12 23:33:17.114651392,48.149090,17.091725,12.105622,7.352591,19.458214,2024-08-12 12:03:12.054394368,3.000497,0.285654,6.636150,2024.105642,12.855927,8.471094,17.341123,1.656176,12.370351,30.150411,1016.768809,270.034358
min,2023-01-01 00:00:00,48.110442,16.969000,0.000000,0.000000,0.000000,2023-01-01 00:00:00,0.000000,0.000000,1.000000,2023.000000,-5.700000,-9.100000,-3.400000,0.000000,1.900000,11.100000,990.100000,0.000000
25%,2023-11-30 16:00:00,48.136692,17.073272,0.000000,0.000000,0.000000,2023-11-30 00:00:00,1.000000,0.000000,4.000000,2023.000000,6.100000,2.500000,10.000000,0.000000,8.300000,24.100000,1011.700000,82.000000
50%,2024-08-28 11:00:00,48.140819,17.116058,2.000000,1.000000,4.000000,2024-08-28 00:00:00,3.000000,0.000000,7.000000,2024.000000,12.600000,8.700000,17.200000,0.000000,11.500000,29.600000,1016.300000,241.000000
75%,2025-05-06 05:00:00,48.162144,17.127928,14.000000,6.000000,23.000000,2025-05-06 00:00:00,5.000000,1.000000,10.000000,2025.000000,19.400000,14.300000,24.900000,0.900000,15.500000,35.200000,1021.700000,422.000000
max,2025-12-20 23:00:00,48.210650,17.138556,583.000000,645.000000,852.000000,2025-12-20 00:00:00,6.000000,1.000000,12.000000,2025.000000,28.900000,23.900000,36.000000,37.300000,34.400000,72.200000,1043.300000,792.000000
std,NaN,0.025700,0.053290,23.826972,19.272692,38.799616,NaN,1.999181,0.451726,3.336885,0.804294,8.016305,7.257390,9.214170,4.179353,5.291168,8.687983,7.980022,212.018349


In [238]:
data.isnull().sum()


datetime                0
nazov                   0
zemepisna_sirka         0
zemepisna_dlzka         0
smer_do                 0
smer_z                  0
pocet_do             8445
pocet_z              8445
was_imputed             0
pocet_total          8445
date                    0
weekday                 0
is_weekend              0
month                   0
year                    0
season                  0
tavg                    0
tmin                    0
tmax                    0
prcp                    0
wspd                    0
wpgt                    0
pres                    0
tsun               121773
dtype: int64

In [239]:
# Uloženie finálneho datasetu
data.to_csv('../Data/final_data.csv', index=False)